# Rental Price Prediction


In this project we will try to predict the price of rental apartments in Berlin by extracting data via Selenium

In [2]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException 
import time
from geopy.geocoders import Nominatim
from googletrans import Translator

import winsound
import pywhatkit


# Crawling

At this step we go for each page and take the urls of the uloaded apartments

In [21]:
def get_urls(page_number):
    page_number = str(page_number)
    web = 'https://www.wg-gesucht.de/1-zimmer-wohnungen-und-wohnungen-in-Berlin.8.1+2.1.'+page_number+'.html?pagination=1&pu='
    PATH = r'C:\Program Files (x86)\chromedriver.exe'
    driver_url = webdriver.Chrome(PATH)
    driver_url.get(web)
    time.sleep(10)
    try:
        driver.find_element_by_xpath("//a[@class='cmpboxbtn cmpboxbtnyes']").click()
    except:
        pass

     #capcha alret
    try:
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe[name^='a-'][src^='https://www.google.com/recaptcha/api2/anchor?']")))
        #WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//span[@id='recaptcha-anchor']"))).click()
        print('capcha')
        winsound.Beep(1000, 500)
        winsound.Beep(1000, 500)
        winsound.Beep(1000, 500)
        pywhatkit.sendwhatmsg('+9720545488595', 'Capcha', int(time.ctime()[11:13]), int(time.ctime()[14:16]) + 2 ,tab_close=False)
        input("Press Enter to continue...")
    except:
        pass
    
    
    urls = []
    url_list = driver_url.find_elements_by_xpath('//h3[@class="truncate_title noprint"]/a')

    for row in url_list:
        urls.append(row.get_attribute('href'))
    driver_url.quit()
    return urls

here we get all the parameters for the urls that we took
we are taking data from German site so we use Translator() to translate to English

also the site have Capcha v2 which you need to solve it by pressing the right pictures, so if there would be a Capcha it will use winsound.Beep which will beep 3 times and also send me Whatsup message with pywhatkit.sendwhatmsg if im not next to the computer

I learned Selenium from two youtubers : 
1) https://www.youtube.com/watch?v=lTypMlVBFM4&t=245s&ab_channel=JohnWatsonRooney

2) https://www.youtube.com/watch?v=Xjv1sY630Uc&ab_channel=TechWithTim

In [4]:
apartment_url = []
size = []
price = []
numOfRooms = []
buildingCond = []
floor = []
hasKitchen = []
heatingType = []
longitude = []
latitude = []
bus_walk = []
parking = []


#folder:
hasWashingMachine = []
hasBalcony = []
hasElevetor = []
###

geolocator = Nominatim(user_agent='myapplication')
translator = Translator()


PATH = r'C:\Program Files (x86)\chromedriver.exe'
driver = webdriver.Chrome(PATH)

for page in range(0,100):
    print(f'page : {page+1}')
    urls = get_urls(page)
    for url in urls:
        if "sort_order" in url: 
            continue
        print(url)
        apartment_url.append(url)
        driver.get(url)
        time.sleep(5)
        
        #accept cookies
        try:
            driver.find_element_by_xpath("//a[@class='cmpboxbtn cmpboxbtnyes']").click()
        except Exception:
            pass
        
        
        #capcha alret
        try:
            WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe[name^='a-'][src^='https://www.google.com/recaptcha/api2/anchor?']")))
            #WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//span[@id='recaptcha-anchor']"))).click()
            print('capcha')
            winsound.Beep(1000, 500)
            winsound.Beep(1000, 500)
            winsound.Beep(1000, 500)
            pywhatkit.sendwhatmsg('+9720545488595', 'Capcha', int(time.ctime()[11:13]), int(time.ctime()[14:16]) + 2 ,tab_close=False)
            input("Press Enter to continue...")    
        except:
            pass 
            

        try:
            size.append(driver.find_elements_by_xpath("//label[@class='amount']")[0].text)
            numOfRooms.append(driver.find_elements_by_xpath("//label[@class='amount']")[1].text)
            price.append(driver.find_elements_by_xpath("//label[@class='amount']")[2].text)
        except NoSuchElementException:
            size.append(None)
            numOfRooms.append(None)
            price.append(None)
     


    #floor
        try:
            child= driver.find_element_by_xpath("//span[@class='mdi mdi-office-building mdi-36px noprint']")
            parent= child.find_element_by_xpath("..")
            translated = translator.translate(parent.get_attribute("innerText").split('\n')[1])
            floor.append(translated.text)
        except NoSuchElementException:
            floor.append(None)
    #kitchen        
        try:
            child= driver.find_element_by_xpath("//span[@class='mdi mdi-silverware-fork-knife mdi-36px noprint']")
            parent= child.find_element_by_xpath("..")
            translated = translator.translate(parent.get_attribute("innerText").split('\n')[1])
            hasKitchen.append(translated.text)
        except NoSuchElementException:
            hasKitchen.append(None)

    #heating        
        try:
            child= driver.find_element_by_xpath("//span[@class='mdi mdi-fire mdi-36px noprint']")
            parent= child.find_element_by_xpath("..")
            translated = translator.translate(parent.get_attribute("innerText").split("\n")[1])
            heatingType.append(translated.text)
        except NoSuchElementException:
            heatingType.append(None)  


    #folder:
        try:
            child= driver.find_element_by_xpath("//span[@class='mdi mdi-folder mdi-36px noprint']")
            parent= child.find_element_by_xpath("..")
            folder = parent.get_attribute("innerText").split("\n")[1]
            if 'Waschmaschine' in folder:
                hasWashingMachine.append(1)
            else:
                hasWashingMachine.append(0)
            if 'Balkon' in folder:
                hasBalcony.append(1)
            else:
                hasBalcony.append(0)
            if 'Aufzug' in folder:
                hasElevetor.append(1)
            else:
                hasElevetor.append(0)
        except NoSuchElementException:
            print("no folder")
            hasWashingMachine.append(0)
            hasBalcony.append(0)
            hasElevetor.append(0)

    #location:
        try:
            adress = driver.find_element_by_xpath("//div[@class='col-sm-4 mb10']/a")
            adress = adress.get_attribute("innerText").replace('\n',' ')
            location = geolocator.geocode(adress)
            longitude.append(location.longitude)
            latitude.append(location.latitude)
        except:
            longitude.append(None)
            latitude.append(None)
            
            
            
    #bus walk:
        try:
            child= driver.find_element_by_xpath("//span[@class='mdi mdi-bus mdi-36px noprint']")
            parent= child.find_element_by_xpath("..")
            bus_walk.append(int(parent.get_attribute("innerText").split('\n')[1].split(' ')[0]))
        except NoSuchElementException:
            bus_walk.append(None) 
    
    # parking:
        try:
            child= driver.find_element_by_xpath("//span[@class='mdi mdi-car mdi-36px noprint']")
            parent= child.find_element_by_xpath("..")
            parking.append(1)
        except NoSuchElementException:
            parking.append(0)
            


            

driver.quit()
   
    
        

In [6]:
df = pd.DataFrame(data = {'size':size,'rooms':numOfRooms,'price':price,'floor':floor,'kitchen':hasKitchen,
                          'heating Type':heatingType,'longitude':longitude,'latitude':latitude,'WashingMach':hasWashingMachine,
                          'Balcony':hasBalcony,'Elevetor':hasElevetor,'bus_walking':bus_walk,'parking':parking,'url':apartment_url})

df.to_csv(r"C:\Users\Kostya\Desktop\Apartments\apartments1.csv")